In [22]:
print(__doc__)
import time
start_time = time.time()
from copy import deepcopy, copy
import math
import scipy.io as sio
import shutil
import os
from random import shuffle
import numpy as np
from pylab import *
# from featext2 import *
import matplotlib.pyplot as plt
%matplotlib inline 
#matplotlib qt
# inline (suitable for ipython only, shown inside browser!) or qt (suitable in general, shown in external window!)
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D #, axes3d
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, ParameterGrid, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV, RandomizedLasso
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, f_classif, mutual_info_classif, SelectFdr
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from collections import OrderedDict
import re
import datetime
import urllib
import tarfile
# import joblib
# from joblib import Parallel, delayed, Memory
from tempfile import mkdtemp
import copy_reg
import types
import itertools
from itertools import compress
from collections import Counter

#import multiprocessing
def _pickle_method(m):
    if m.im_self is None:
        return getattr, (m.im_class, m.im_func.func_name)
    else:
        return getattr, (m.im_self, m.im_func.func_name)
copy_reg.pickle(types.MethodType, _pickle_method)

h = .2  # step size in the mesh
names = ["NearNb", "RBFSVM1", "NaiveBayes", "MLP1", "Log.Regr", "RandFor", "AdaBoost", "EnsembleMLP"]
classifiers = [
    KNeighborsClassifier(5),
    SVC(gamma='auto', C=1),
    MLPClassifier(solver='lbfgs',alpha=1e-4,hidden_layer_sizes=(10,10),random_state=1,verbose=True),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
]

cv = StratifiedKFold(n_splits=5, random_state=42)
parameters_clf = [{'n_neighbors':range(3,10)},
              {'kernel':['rbf'], 'C':[0.01,0.1,1,10,100,1000]},
              {'solver':['lbfgs'], 'alpha':[1e-5,1e-2], 'hidden_layer_sizes':[(10,10),(50,50),(100,100)]},
              {'max_depth':[4,7,10,20],'n_estimators':[5,10,20],'max_features':[20,35,50]},
              {'solver':['lbfgs'], 'alpha':[1e-5], 'hidden_layer_sizes':[(len(names)-1,len(names)-1),(len(names)-1,2)]}
             ]
makepipe_parameters_clf = [{'classifier__'+key:p[key] for key in p} for p in parameters_clf]
makepipe_parameters_clf += [{'feature_selection__k': (750,500,100), 'feature_selection__score_func': [mutual_info_classif]},
                            {'decomp__n_components': (100,50)}]
metric = ['accuracy','f1']
dataset = 0 # all datasets (0), dataset 1-2 (1), dataset 3 (2), dataset4 (3)
download = 1 # Download pre-computed (1) data or compute them anew (0)
window = 1024

Automatically created module for IPython interactive environment


In [3]:
def make_pipe_clf(scaler,feature_selection,decomp,clf):
    order = 1
# order = 1 : first perform feature selection and then apply PCA
# order = 0 : first apply PCA and then reduce the transformed features
    if order:
        pipeline = Pipeline([('scaler', scaler),
                    ('feature_selection', feature_selection),
                    ('decomp', decomp),         
                    ('classifier', clf) ])
#     else:
#         pipeline = Pipeline([('scaler', scaler),
#                     ('decomp', decomp ),                 
#                     ('feature_selection', feature_selection),        
#                     ('classifier', clf) ])
    return pipeline
###########################################################################################
def make_pipe(scaler,feature_selection,decomp,order):
    if order:
        pipeline = Pipeline([('scaler', scaler),
                    ('feature_selection', feature_selection),
                    ('decomp', decomp),         
                     ])
    else:
        pipeline = Pipeline([('scaler', scaler),
                    ('decomp', decomp ),                 
                    ('feature_selection', feature_selection),        
                     ])
    return pipeline

In [108]:
def get_feat_id(feat_ind, printit = 0): 
    sample_window = 1024
    ##  features:                                                                                  ||      if         ##
##  |----------> time domain      :                                                            || samples = 1024  ##
##  |------------|---> phinyomark : 11+3{shist} -----------------------------> = 14+0.0samples ||             14  ##
##  |------------|---> golz       : 10+samples{acrol} -----------------------> = 10+1.0samples ||           1034  ##
##  |----------> frequency domain :                                                                               ##
##  |------------|---> phinyomark : 3{arco}+4{mf}+3(samples/2+1){RF,IF} -----> =  9+1.0samples ||           1033  ##
##  |------------|---> golz       : 2(samples/2+1){AF,PF} -------------------> =  2+1.0samples ||           1027  ##
##  |------------|--------|-------alltogether--------------------------------> = 35+3.0samples || numfeat = 3108  ##
    # get the feat inds wrt their source : 3rd level
    norm_time_phin = range(0,14)
    norm_freq_phin = range(norm_time_phin[-1] + 1, norm_time_phin[-1] + 9 + sample_window + 1)
    norm_time_golz = range(norm_freq_phin[-1] + 1, norm_freq_phin[-1] + 10 + sample_window + 1)
    norm_freq_golz = range(norm_time_golz[-1] + 1, norm_time_golz[-1] + 2 + sample_window + 1)
    # get the feat inds wrt their domain : 2nd level 
    norm_time_feats = norm_time_phin + norm_time_golz
    norm_freq_feats = norm_freq_phin + norm_freq_golz
    # get the feat inds wrt their prefeat: 1st level 
    norm_feats = norm_time_feats + norm_freq_feats

    # get the feat inds wrt their source : 3rd level
#     np.arange(norm_feats[-1]+1,norm_feats[-1]+1+len(norm_feats))
    disp = norm_feats[-1]+1
    ftfn_time_phin = range(disp ,disp + 14)
    ftfn_freq_phin = range(ftfn_time_phin[-1] + 1, ftfn_time_phin[-1] + 9 + sample_window + 1)
    ftfn_time_golz = range(ftfn_freq_phin[-1] + 1, ftfn_freq_phin[-1] + 10 + sample_window + 1)
    ftfn_freq_golz = range(ftfn_time_golz[-1] + 1, ftfn_time_golz[-1] + 2 + sample_window + 1)
    # get the feat inds wrt their domain : 2nd level 
    ftfn_time_feats = ftfn_time_phin + ftfn_time_golz
    ftfn_freq_feats = ftfn_freq_phin + ftfn_freq_golz
    # get the feat inds wrt their prefeat: 1st level 
    ftfn_feats = ftfn_time_feats + ftfn_freq_feats

    # create the final "reference dictionary"
    id_list = [np.zeros((len(ftfn_feats + norm_feats),1)) for i in range(3)] #3 np.arrays, id_list[0] = level 1 etc
    id_list[0][:norm_feats[-1]+1] = 0 # 0 signifies norm / 1 signifies ft/fn
    id_list[0][norm_feats[-1]+1:] = 1

    id_list[1][:norm_time_phin[-1]+1] = 0 #0 signifies time / 1 signifies freq
    id_list[1][norm_time_phin[-1]+1:norm_freq_phin[-1]+1] = 1
    id_list[1][norm_freq_phin[-1]+1:norm_time_golz[-1]+1] = 0
    id_list[1][norm_time_golz[-1]+1:norm_freq_golz[-1]+1] = 1
    id_list[1][norm_freq_golz[-1]+1:ftfn_time_phin[-1]+1] = 0
    id_list[1][ftfn_time_phin[-1]+1:ftfn_freq_phin[-1]+1] = 1
    id_list[1][ftfn_freq_phin[-1]+1:ftfn_time_golz[-1]+1] = 0
    id_list[1][ftfn_time_golz[-1]+1:] = 1

    id_list[2][:norm_freq_phin[-1]+1] = 0 #0 signifies phinyomark / 1 signifies golz
    id_list[2][norm_freq_phin[-1]+1:norm_freq_golz[-1]+1] = 1
    id_list[2][norm_freq_golz[-1]+1:ftfn_freq_phin[-1]+1] = 0
    id_list[2][ftfn_freq_phin[-1]+1:] = 1 
    
    full_path_id = [np.zeros((len(feat_ind),5)) for i in range(len(feat_ind))]
   
    for ind, val in enumerate(feat_ind):

        full_path_id[ind] = [val, id_list[2][val], id_list[1][val], id_list[0][val]]

        if (printit==1):
            if(full_path_id[ind][1]==0):
                lvl3 = 'Phin'
            else:
                lvl3 = 'Golz'
            if(full_path_id[ind][2]==0):
                lvl2 = 'Time'
            else:
                lvl2 = 'Freq'
            if(full_path_id[ind][3]==0):
                lvl1 = 'Norm'
            else:
                lvl1 = 'Ft/Fn'
            print(feat_ind[ind],featnames[val%(norm_feats[-1]+1)],lvl3,lvl2,lvl1)
    
    return(full_path_id)        

In [34]:
def get_feat_occ(feat_masks):
    #get the number of occurences for each feature after SelectKbest
#     print("If it ain't working, just make sure you're adding the lists instead of concatenating them,")
#     print("if the input isn't a single list you'll get the unhashable error")
    feat_occ = Counter(feat_masks)
    return feat_occ 

In [6]:
class Nested_cv:
    def __init__(self, n_outer_folds = 3, n_inner_folds = 3, n_top =1, state = 42):
        self.n_outer_folds = n_outer_folds
        self.n_inner_folds = n_inner_folds
        self.n_top = n_top
        self.state = state
#         print("constructor %d" %self.state)
                                        
    def set_pipe(self, pip_steps, pip_params):
        n_steps = len(pip_steps)
        if n_steps == 3:
            self.pipe = make_pipe(scaler = pip_steps[0], feature_selection = pip_steps[1], decomp = pip_steps[2])
            self.params = pip_params
        if n_steps == 4:
            self.pipe = make_pipe_clf(scaler = pip_steps[0], feature_selection = pip_steps[1], decomp = pip_steps[2], clf = pip_steps[3])
            self.params = pip_params
        else:
            print ("Number of steps gotta be either 3 or 4, you inserted %d." %n_steps)
        return self.pipe
        
    def fit(self, x_tot, y_tot, verbose = 0):
        self.verbose = verbose
#         print("fit %d" %self.state)
        if self.n_top >1 :
            print ("ACHTUNG! You need to use fit2 for multiple best models!")
            self.fit2(x_tot,y_tot)
        else:
            self.outer_cv = StratifiedKFold(n_splits = self.n_outer_folds, shuffle = True, random_state = self.state)
            self.inner_cv = StratifiedKFold(n_splits = self.n_inner_folds, shuffle = True, random_state = self.state)
            self.grid = GridSearchCV(self.pipe, self.params, cv = self.inner_cv, verbose = self.verbose)       
            self.score = cross_val_score(self.grid, X = x_tot, y = y_tot, cv = self.outer_cv, verbose = self.verbose)

    def get_outer_scores(self):
        return self.score
    
    def get_mean_score(self):
        self.mean_score = self.score.mean()
        print ("Mean score of %d outer folds : %f" %(self.n_outer_folds, self.mean_score ))
        return self.mean_score
    
#     def combinations(self):
#         self.comb = 1
#         for self.key in self.params.keys():
#             self.comb =self.comb*len(self.params[self.key])
#         return self.comb
    
    def get_feat_scores(self):
        return self.total_feats
    
    def get_best_features(self):
        print(self.best_feat_ind)
        return self.best_feat_ind
    
    def print_feat_scores(self):
        self.norm_total_feats = (self.total_feats)/(self.n_inner_folds * self.n_top)
        plt.figure()
        self.rel_score = plt.bar(range(len(self.total_feats)),self.norm_total_feats)
        plt.xlabel("Features")
        plt.ylabel("Score")
        plt.show()
        
    
    def repeated_cv(self,x_tot, y_tot, num_trials):
        self.repeated = np.zeros(num_trials)
        self.rep_feat_scores = np.zeros(x_tot.shape[1])
        self.num_trials = num_trials
        self.state +=1
        for st in range(self.num_trials):
            self.state +=1
            self.fit(x_tot, y_tot)
#             print("repeated %d" %self.state)
            self.out = self.get_outer_scores()
            self.rep_feat_scores += self.get_feat_scores()
            self.repeated[st] = self.out.mean()
            
        plt.figure()
        self.rep_sc_plot = plt.bar(range(num_trials),self.repeated)
        plt.xlabel("Individual trial #")
        plt.ylabel("Outer mean scores")
        plt.show()
    
    def print_rep_feat_scores(self):
        plt.figure()
        self.norm_rep_feats = (self.rep_feat_scores)/(self.n_inner_folds*self.n_top*self.num_trials)
        self.rel_score = plt.bar(range(len(self.total_feats)),self.norm_rep_feats)
        plt.xlabel("Features")
        plt.ylabel("Feature Score")
        plt.show()
        
    def get_rep_feat_scores(self):
        return self.rep_feat_scores
        
    def fit2(self,x_tot, y_tot):
        #the "handwritten" implementation
        self.total_feats = np.zeros(x_tot.shape[1])
        self.classifiers= list(ParameterGrid(self.params))
        self.outer_scores = np.zeros((self.n_top, self.n_outer_folds))
#         self.ncomp = self.params['decomp__n_components']
#         self.best_feat_ind = []
        self.best_pipes = [[],[]]
        self.outer_cv = StratifiedKFold(n_splits = self.n_outer_folds, shuffle = True, random_state = self.state)
        self.inner_cv = StratifiedKFold(n_splits = self.n_inner_folds, shuffle = True, random_state = self.state)
        self.outer = self.outer_cv.split(x_tot,y_tot)
        for self.fold_out, (self.train_ind_out,self.test_ind_out) in enumerate(self.outer):
            top_models = []
            if self.verbose>0 :
                print("Outer loop %d/%d" %((self.fold_out + 1), self.n_outer_folds))
        # split the dataset 
            self.x_trn_out, self.x_tst_out = x_tot[self.train_ind_out], x_tot[self.test_ind_out]
            self.y_trn_out, self.y_tst_out = y_tot[self.train_ind_out], y_tot[self.test_ind_out]  
            
            self.inner_scores = np.zeros((len(self.classifiers),self.n_inner_folds))
            self.inner_mean_scores = np.zeros(len(self.classifiers))
            
            for self.cl_ind, self.clf in enumerate(self.classifiers):
                self.inner = self.inner_cv.split(self.x_trn_out, self.y_trn_out)
                self.nfeat = self.classifiers[self.cl_ind]['feature_selection__k']
                self.best_feat_ind = np.zeros((self.n_top, self.nfeat))


                for self.fold_in, (self.train_ind_in, self.test_ind_in) in enumerate(self.inner): 
                    if self.verbose>0 :
                        print("Inner fold %d/%d" % ((self.fold_in + 1), self.n_inner_folds ))
                    # split the datasets for the inner cv 
                    self.x_trn_in, self.x_tst_in = self.x_trn_out[self.train_ind_in], self.x_trn_out[self.test_ind_in]
                    self.y_trn_in, self.y_tst_in = self.y_trn_out[self.train_ind_in], self.y_trn_out[self.test_ind_in]
                    self.pip = self.pipe.set_params(**self.clf)
                    self.pip.fit(self.x_trn_in,self.y_trn_in)
                    self.inner_scores[self.cl_ind][self.fold_in] = self.pip.score(self.x_tst_in, self.y_tst_in)

            # get the mean performance for every classifier
            self.inner_mean_scores = np.mean(self.inner_scores, axis = 1)
#             print('\n',self.inner_mean_scores)

            # sort the scores, low -> high 
            self.sorted_inds = self.inner_mean_scores.argsort()
            self.sorted_scores = self.inner_mean_scores[self.sorted_inds]

            print ("Best %d models for outer fold %d are: \n " %(self.n_top, (self.fold_out+1)))

#             get the inds of best performing models
            self.temp2 = 0 #iterates over n_top models
            for self.temp_ind in range(-1, -(self.n_top +1), -1):
                self.actual_ind = self.sorted_inds[self.temp_ind]
                self.best_pipes[1].append(self.classifiers[self.actual_ind]) 
                self.best_pipes[0].append(self.fold_out)
                print("model no.%d "%(self.actual_ind + 1))
                print(self.classifiers[self.actual_ind])
                #print the top features selected
                self.best_fs = self.pipe.set_params(**self.classifiers[self.actual_ind]).named_steps['feature_selection']
#                 best_fs = pipeline[actual_ind].named_steps['feature_selection']
                self.best_fs_indd = self.best_fs.get_support(indices = True)
#                 print(self.best_fs_indd)
                self.pipe_fs_unsorted = self.best_fs.scores_
                self.fs_inds = self.pipe_fs_unsorted.argsort()
#                 print(self.fs_inds)
                self.pipe_fs = self.pipe_fs_unsorted[self.fs_inds]
                self.temp3 = 0 # temp3 = 0:number of features
               
                for self.temp_ind2 in range(-1, -(self.nfeat +1), -1):         
                    self.best_feat_ind[self.temp2][self.temp3]=self.fs_inds[self.temp_ind2]
                    self.total_feats[self.fs_inds[self.temp_ind2]]+=1 # feature scores 
                    self.temp3+=1
#                 print(self.total_feats)
                #fit the best classifier on the outer test data
                self.best_pip = self.pipe.set_params(**self.classifiers[self.actual_ind])
                
                self.best_pip.fit(self.x_trn_out, self.y_trn_out)
                
                #get the outer score 
                self.outer_scores[self.temp2, self.fold_out] = self.best_pip.score(self.x_tst_out, self.y_tst_out)
                print("Inner score: %f VS Outer score: %f \n" %(self.inner_mean_scores[self.actual_ind], self.outer_scores[self.temp2, self.fold_out]))
                print ("----------------------------------------------------------")
                self.temp2+=1


#             print("Features selected: \n")
        #     print(best_feat_ind.sort())     
#             print(self.best_feat_ind)
        # print(outer_scores)  
        # print(best_pipes)
        self.score = self.outer_scores

In [7]:
########################################## Feature Names ###########################################################
####################################################################################################################
##  features:                                                                                  ||      if         ##
##  |----------> time domain      :                                                            || samples = 1024  ##
##  |------------|---> phinyomark : 11+3{shist} -----------------------------> = 14+0.0samples ||             14  ##
##  |------------|---> golz       : 10+samples{acrol} -----------------------> = 10+1.0samples ||           1034  ##
##  |----------> frequency domain :                                                                               ##
##  |------------|---> phinyomark : 3{arco}+4{mf}+3(samples/2+1){RF,IF} -----> =  9+1.0samples ||           1033  ##
##  |------------|---> golz       : 2(samples/2+1){AF,PF} -------------------> =  2+1.0samples ||           1027  ##
##  |------------|--------|-------alltogether--------------------------------> = 35+3.0samples || numfeat = 3108  ##
####################################################################################################################
## Time Domain Phinyomark feats
featnames = ['intsgnl', 'meanabs', 'meanabsslp', 'ssi', 'var', 'rms', 'rng', 'wavl', 'zerox', 'ssc', 'wamp', 
             'shist1', 'shist2', 'shist3']                                                   # 11+3{shist}
## Frequency Domain Phinyomark feats
featnames += ['arco1', 'arco2', 'arco3', 'mnf', 'mdf', 'mmnf', 'mmdf']                       # 3{arco}+4{mf}
featnames += ['reFFT{:03d}'.format(i) for i in range(window/2+1)]                            # samples/2+1{RF}
featnames += ['imFFT{:03d}'.format(i) for i in range(window/2+1)]                            # samples/2+1{IF}
## Time Domain Golz feats
featnames += ['meanv', 'stdr', 'mx', 'rngx', 'rngy', 'med', 'hjorth', 'sentr', 'se', 'ssk']  # 10
featnames += ['acrol{:04d}'.format(i) for i in range(window)]                                # samples{acrol}
## Frequency Domain Golz feats
featnames += ['amFFT{:03d}'.format(i) for i in range(window/2+1)]                            # samples/2+1{AF}
featnames += ['phFFT{:03d}'.format(i) for i in range(window/2+1)]                            # samples/2+1{PF}
# featnames += ['ffaf']                                                                        # 1{ffaf}

In [8]:
datasets = np.load('newfeatures_newdata_NOsample_1024_20_10_10000_XYsplit.npz')
X = datasets['Xsp']
Y = datasets['Ysp']

In [11]:
print([ X[i][j].shape for i in range(len(X)) for j in range(len(X[i]))])

[(8902, 3107), (8902, 3107), (8902, 6214), (8902, 3107), (8902, 3107), (8902, 6214), (8898, 3107), (8898, 3107), (8898, 6214)]


In [7]:
# i: iterating over finger, j: over featureset 
# (consider keeping only the second featureset and discarding the rest to save some space)
# Performing cross validation for each finger on the second feature set 

cv = KFold(n_splits=5,random_state=42)
# set the pipeline
# def make_pipe_clf(scaler,feature_selection,decomp,clf):
scaler = StandardScaler() ; featsel = SelectKBest(k=1000,score_func= mutual_info_classif)
decomp = PCA(n_components=20)
pipe_list = [make_pipe_clf(scaler,featsel, decomp, classifiers[i]) for i in range(len(classifiers))]
scores = []
for i in range(len(X)):
    data = deepcopy(X[i][2])
    print(i)
    labels = deepcopy(Y[i])
    for pipe in pipe_list:
        score = cross_val_score(estimator = pipe, X = data, y = labels, cv = cv, verbose = 100)
        scores.append(score)
        print ("=============================================================")
        print("Datasets : %d, classifier : %0.4s, 5-fold avg score = %f" %(i, pipe.named_steps['classifier'], np.mean(score)))
        print(score)
        print ("=============================================================")
        
# print pipe_list[1].named_steps['classifier']
    
    

0
[CV]  ................................................................
[CV] ................................. , score=0.895564, total= 6.5min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.5min remaining:    0.0s
[CV]  ................................................................


KeyboardInterrupt: 

In [4]:
# cross-validate on the different objects using the dataset from both fingers
data_X = deepcopy(X[2][2]) # choose the featureset
data_Y = deepcopy(Y[2])
surfaces = np.split(data_X,3)
surf_labels = np.split(data_Y,3)

cv = KFold(n_splits=5,random_state=42)
scaler = StandardScaler() ; featsel = SelectKBest(k=1000,score_func= mutual_info_classif)
decomp = PCA(n_components=20)
pipe_list = [make_pipe_clf(scaler,featsel, decomp, classifiers[i]) for i in range(len(classifiers))]
scores = []

for surf_ind, surf_data in enumerate(surfaces):
    print(surf_ind)
    data = surf_data
    labels = surf_labels[surf_ind]
    for pipe in pipe_list:
        score = cross_val_score(estimator = pipe, X = data, y = labels, cv = cv, verbose = 100)
        scores.append(score)
        print ("=============================================================")
        print("Object : %d, classifier : %0.4s, 5-fold avg score = %f" %(surf_ind, pipe.named_steps['classifier'], np.mean(score)))
        print(score)
        print ("=============================================================")
        

0
[CV]  ................................................................
[CV] ................................. , score=0.887205, total= 2.5min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.924115, total= 2.4min
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.9min remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.905565, total= 2.4min
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  7.3min remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.934233, total= 2.4min
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  9.7min remaining:    0.0s
[CV]  ................................................................
[CV] ................................. 

[CV] ................................. , score=0.784512, total= 2.3min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.3min remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.967960, total= 2.3min
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.6min remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.777403, total= 2.5min
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  7.0min remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.917369, total= 2.5min
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  9.5min remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.920742, total= 2.4min
[Parallel(n_jobs=1)]: Done   5 out of   5

In [33]:
%%time
# validate between the different objects using the dataset from both fingers
# Dataset used : Both fingers 
# Featureset used : Ft/Fn
# Version 1: keep alternatively one for training and test on the rest


data_X = deepcopy(X[2][1]) 
data_Y = deepcopy(Y[2])
surfaces = np.split(data_X,3)
surf_labels = np.split(data_Y,3) 
feat_mask = []

scaler = StandardScaler() ; featsel = SelectKBest(k=1000,score_func= mutual_info_classif)
decomp = PCA(n_components=20)
pipe_list = [make_pipe_clf(scaler,featsel, decomp, classifiers[i]) for i in range(len(classifiers))]

for surf_ind, surf_dat in enumerate(surfaces):
    print("Training on surface no. %d" %surf_ind)
    ind_mask = [True, True, True]
    ind_mask[surf_ind] = False #  the training dataset is flagged by False
    train_x = surf_dat
    train_y = surf_labels[surf_ind]
#     print(ind_mask)
    test_x = list(compress(surfaces, ind_mask)) # the rest splits, flagged by True, are kept for testing
    test_y = list(compress(surf_labels,ind_mask))
    
    for pipe_ind,pipe in enumerate(pipe_list):
        print("Fitting classifier no. %d..." %pipe_ind)
        pipe.fit(train_x,train_y) # fit the pipeline for every train set and every clf
        print ("...done fitting")
        feat = list(pipe.named_steps['feature_selection'].get_support(indices = True))
#         print(feat[:20])
        feat_mask+=feat
#         print("done getting features")
#         print(get_feat_occ(feat))
#         print("lets print those fuckers")
#         get_feat_id(feat[:20], printit=1)
        
        for test_ind, test_d in enumerate(test_x):
            y_pred = pipe.predict(test_d)
            y_true = test_y[test_ind]
            cm = confusion_matrix(y_pred=y_pred, y_true=y_true)
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            print ("=============================================================")
            print("Trained on %d, testing on %d with %0.4s" %(surf_ind, ind_mask.index("True") pipe.named_steps['classifier']))
            print("Prediction accuracy %f" %pipe.score(test_d,y_true))
            print(cm)
            print(classification_report(y_pred=y_pred, y_true = y_true))        
            print ("=============================================================")
    
tot_occs = get_feat_occ(feat_mask)
# ids = get_feat_id(tot_occs[:20],printit=1)
        


Training on surface no. 0
Fitting classifier no. 0...
...done fitting
Trained on 0 with KNei
[[ 0.92158134  0.07841866]
 [ 0.20660576  0.79339424]]
             precision    recall  f1-score   support

        0.0       0.83      0.92      0.87      1543
        1.0       0.90      0.79      0.84      1423

avg / total       0.86      0.86      0.86      2966

Trained on 0 with KNei
[[ 0.92875318  0.07124682]
 [ 0.28766141  0.71233859]]
             precision    recall  f1-score   support

        0.0       0.78      0.93      0.85      1572
        1.0       0.90      0.71      0.79      1394

avg / total       0.84      0.83      0.82      2966

Fitting classifier no. 1...
...done fitting
Trained on 0 with SVC(
[[ 0.96759559  0.03240441]
 [ 0.14617006  0.85382994]]
             precision    recall  f1-score   support

        0.0       0.88      0.97      0.92      1543
        1.0       0.96      0.85      0.90      1423

avg / total       0.92      0.91      0.91      2966

Trained

Trained on 2 with SVC(
[[ 0.78742709  0.21257291]
 [ 0.04848911  0.95151089]]
             precision    recall  f1-score   support

        0.0       0.95      0.79      0.86      1543
        1.0       0.80      0.95      0.87      1423

avg / total       0.88      0.87      0.87      2966

Fitting classifier no. 2...
...done fitting
Trained on 2 with MLPC
[[ 0.8739386  0.1260614]
 [ 0.1184669  0.8815331]]
             precision    recall  f1-score   support

        0.0       0.89      0.87      0.88      1531
        1.0       0.87      0.88      0.87      1435

avg / total       0.88      0.88      0.88      2966

Trained on 2 with MLPC
[[ 0.8250162   0.1749838 ]
 [ 0.09908644  0.90091356]]
             precision    recall  f1-score   support

        0.0       0.90      0.83      0.86      1543
        1.0       0.83      0.90      0.86      1423

avg / total       0.86      0.86      0.86      2966

Fitting classifier no. 3...
...done fitting
Trained on 2 with Rand
[[ 0.87067276 

In [117]:
from collections import OrderedDict
# feats = [tot_occs[i] for i in range(40) ]
# print(tot_occs)
# print(tot_occs)
# tot = dict(tot_occs)
feat_list = []
asd = OrderedDict(sorted(tot_occs.items(), key=lambda tot_occs: tot_occs[1]))
for k,v in asd.items()[200:505]:
    feat_list.append(k)
nio = get_feat_id(feat_list,printit = 1)
# print feat_list

(1404, 'acrol0347', 'Golz', 'Time', 'Norm')
(1405, 'acrol0348', 'Golz', 'Time', 'Norm')
(1406, 'acrol0349', 'Golz', 'Time', 'Norm')
(1442, 'acrol0385', 'Golz', 'Time', 'Norm')
(1473, 'acrol0416', 'Golz', 'Time', 'Norm')
(1482, 'acrol0425', 'Golz', 'Time', 'Norm')
(1483, 'acrol0426', 'Golz', 'Time', 'Norm')
(1487, 'acrol0430', 'Golz', 'Time', 'Norm')
(1490, 'acrol0433', 'Golz', 'Time', 'Norm')
(1496, 'acrol0439', 'Golz', 'Time', 'Norm')
(1497, 'acrol0440', 'Golz', 'Time', 'Norm')
(1498, 'acrol0441', 'Golz', 'Time', 'Norm')
(1499, 'acrol0442', 'Golz', 'Time', 'Norm')
(1500, 'acrol0443', 'Golz', 'Time', 'Norm')
(1507, 'acrol0450', 'Golz', 'Time', 'Norm')
(1508, 'acrol0451', 'Golz', 'Time', 'Norm')
(1511, 'acrol0454', 'Golz', 'Time', 'Norm')
(1513, 'acrol0456', 'Golz', 'Time', 'Norm')
(1515, 'acrol0458', 'Golz', 'Time', 'Norm')
(1517, 'acrol0460', 'Golz', 'Time', 'Norm')
(1518, 'acrol0461', 'Golz', 'Time', 'Norm')
(1519, 'acrol0462', 'Golz', 'Time', 'Norm')
(1520, 'acrol0463', 'Golz', 'Tim

In [41]:
surfaces = np.split(data_X,3)
print([surfaces[i][1,0] for i in range(3)])

[439.91478874556327, 623.21723828234553, 833.93077225598097]


In [ ]:
state = 42
skipdata = 20
best_parameters = np.zeros((X.shape[0],len(tmpind),len(classifiers))).tolist()
best_clf = np.zeros((X.shape[0],len(tmpind),len(classifiers))).tolist()
best_fs_sc = np.zeros((X.shape[0],len(tmpind),len(classifiers))).tolist()
masks = []
for i in range(X.shape[0]):
    y_tot = np.copy(Ysp[i]) ; y_tot = y_tot[::skipdata]
    for j in range(len(tmpind)):
        x_tot = np.copy(Xsp[i][j]) ; x_tot = x_tot[::skipdata,:]
#         print(x_tot.shape)
        fs = SelectKBest(k=100)
        fs.fit(x_tot,y_tot)
        mask = list(fs.get_support(indices = True))
        masks+=mask
#         print masks
occs = get_feat_occ(masks)
print(occs)   

In [16]:
ind_list = [1,0,1]
bool_ind = [bool(ind_list[i]) for i in range(len(ind_list))]
bool_result = X[bool_ind]
print (len(X),len(bool_result))
print([ bool_result[i][j].shape for i in range(len(bool_result)) for j in range(len(bool_result[i]))])
print([ X[i][j].shape for i in range(len(X)) for j in range(len(X[i]))])

(3, 2)
[(8902, 3107), (8902, 3107), (8902, 6214), (8898, 3107), (8898, 3107), (8898, 6214)]
[(8902, 3107), (8902, 3107), (8902, 6214), (8902, 3107), (8902, 3107), (8902, 6214), (8898, 3107), (8898, 3107), (8898, 6214)]
